# Context
- Show the test list
- Predict single one
    - spectrum
    - NMO result
    - VMAE

In [1]:
####################### Import Settings #####################
import argparse
import os
import warnings

import h5py
import numpy as np
import pandas as pd
import segyio
import torch

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from utils.evaluate import GetResult
from net.MIFNet import MultiInfoNet
from utils.LoadData import LoadSingleData, PredictSingleLoad

warnings.filterwarnings("ignore")

In [2]:
############### Base Setting ###############################
parser = argparse.ArgumentParser()
parser.add_argument('--DataSet', type=str, default='hade', help='Dataset Root Path')
parser.add_argument('--DataSetRoot', type=str, help='Dataset Root Path')
parser.add_argument('--GatherLen', type=int, default=21)
parser.add_argument('--LoadModel', type=str, help='Model Path')
parser.add_argument('--SGSMode', type=str, default='all')
parser.add_argument('--Predthre', type=float, default=0.3)
parser.add_argument('--SeedRate', type=float, default=0.5)
parser.add_argument('--Resave', type=int, default=0)
parser.add_argument('--GPUNO', type=int, default=0)
parser.add_argument('--Resize', type=list, help='CropSize')
parser.add_argument('--SizeH', type=int, default=256, help='Size Height')
parser.add_argument('--t0Int', type=list, default=[])
opt = parser.parse_args(args=[])
# setting
opt.DataSetRoot = 'E:\\Spectrum\\hade'
opt.LoadModel = 'F:\\VSP-MIFN\\0Ablation\\hade\\model\\DS_hade-SR_0.80-LR_0.0010-BS_8-SH_256-PT_0.10-SGSM_mute.pth'
opt.Predthre = 0.3
opt.SeedRate = 0.5

In [3]:
########## Show the test index ###################
# load segy data
SegyName = {'pwr': 'vel.pwr.sgy',
            'stk': 'vel.stk.sgy',
            'gth': 'vel.gth.sgy'}
SegyDict = {}
for name, path in SegyName.items():
    SegyDict.setdefault(name, segyio.open(os.path.join(opt.DataSetRoot, 'segy', path), "r", strict=False))
# load h5 file
H5Name = {'pwr': 'SpecInfo.h5',
            'stk': 'StkInfo.h5',
            'gth': 'GatherInfo.h5'}
H5Dict = {}
for name, path in H5Name.items():
    H5Dict.setdefault(name, h5py.File(os.path.join(opt.DataSetRoot, 'h5File', path), 'r'))

# load label.npy
LabelDict = np.load(os.path.join(opt.DataSetRoot, 't_v_labels.npy'), allow_pickle=True).item()
HaveLabelIndex = []
for lineN in LabelDict.keys():
    for cdpN in LabelDict[lineN].keys():
        HaveLabelIndex.append('%s_%s' % (lineN, cdpN))
pwr_index = set(H5Dict['pwr'].keys())
stk_index = set(H5Dict['stk'].keys())
gth_index = set(H5Dict['gth'].keys())

# find common index
Index = sorted(list((pwr_index & stk_index) & (gth_index & set(HaveLabelIndex))))
trainIndex, testIndex = train_test_split(Index, test_size=0.2, random_state=123)
trainIndex, _ = train_test_split(trainIndex, test_size=1-opt.SeedRate, random_state=123)

# Print test list
print('Predict Number', len(testIndex), '\n', testIndex)

Predict Number 217 
 ['2600_880', '2320_960', '3120_2360', '2840_960', '2560_1680', '2520_1200', '2600_1600', '3120_920', '2320_2280', '2960_2240', '2680_1680', '3200_2000', '2760_880', '3200_1120', '2880_2360', '3160_1800', '2520_2400', '2560_2360', '2640_2280', '2360_800', '2600_1720', '2240_1960', '2920_1720', '3120_1120', '3240_1040', '2760_2200', '3040_920', '2400_2000', '2360_1280', '2440_1560', '2480_2400', '2760_2360', '3240_880', '2560_1080', '2720_2160', '2280_2200', '2400_800', '2400_2320', '3240_960', '2480_2080', '3000_1000', '3200_2360', '3200_1200', '2720_1360', '2320_1600', '3120_1240', '2880_2200', '2400_1880', '2560_1600', '2480_1160', '3000_1720', '3080_1600', '2680_1720', '3000_920', '2440_1080', '3200_840', '2360_1040', '2520_920', '2920_1120', '2440_1320', '3160_1840', '2440_1000', '3120_1360', '2480_1960', '3080_1920', '2880_1280', '3200_1920', '3240_2120', '2400_1240', '3080_1040', '2840_1040', '2960_1520', '2800_1720', '3120_1080', '2920_2080', '2480_1800', '26

In [4]:
############### Load Model ####################################
# Load MIFN
opt.t0Int = np.array(SegyDict['pwr'].samples)
opt.Resize = [int(opt.SizeH), int(opt.SizeH/2)]
net = MultiInfoNet(opt.t0Int, opt, in_channels=11, resize=opt.Resize)
use_gpu = torch.cuda.device_count() > 0
if use_gpu:
    net = net.cuda(opt.GPUNO)
net.eval()
# Load Model
net.load_state_dict(torch.load(opt.LoadModel)['Weights'])

<All keys matched successfully>

In [5]:
############### Predict Single One #################################
index = '2800_2040'
# Load data
DataDict = LoadSingleData(SegyDict, H5Dict, LabelDict, index, mode='train', LoadG=True)
VInt = DataDict['vInt']
FM, stkG, stkC, mask, VMM, ManualCurve = PredictSingleLoad(DataDict, opt.t0Int, opt.Resize, opt.GatherLen)
print(list(DataDict.keys()))
if use_gpu:
    pwr = FM.cuda(opt.GPUNO)
    stkG = stkG.cuda(opt.GPUNO)
    stkC = stkC.cuda(opt.GPUNO)

out, STKInfo = net(pwr, stkG, stkC, VMM)
PredSeg, STKInfo = out.squeeze(), STKInfo.squeeze()

['spectrum', 'vInt', 'stkG', 'stkC', 'label']


# Single Predict
## Prediction Map
PredSeg is the probability prediction segment map for one sample.


In [ ]:
####### Visual the result ##########
